# Clasificación de tumores sobre datos de expresión génica obtenidos por RNA-Seq con técnicas de aprendizaje máquina

## Resumen

En este trabajo se plantea la utilización de diversas técnicas de machine learning con el fin de resolver un problema de clasificación que permita etiquetar distintos tipos de tumores en función de valores de expresión de diversos genes obtenidos por medio de la técnica de RNA-Seq en numerosos pacientes. Dada la enorme cantidad de atributos de los datos de entrada, en primer lugar es necesario un preprocesamiento de la información para más tarde poder entrenar distintos tipos de modelos (en este caso: algoritmo kNN, máquina de soporte vectorial y random forest). Los resultados muestran que los datos se pueden procesar y clasificar de forma satisfactoria.

## Introducción

El presente trabajo, de temática biomédica, se fundamenta en la utilización de diversas técnicas de machine learning para la consecución de un problema de clasificación. Para ello se dispone de una base de datos procedente del repositorio "UCI Machine Learning" denominada **gene expression cancer RNA-Seq Data Set** (disponible en la siguiente URL https://archive.ics.uci.edu/ml/datasets/gene+expression+cancer+RNA-Seq#). Esta, contiene instancias de niveles de expresión génica extraídos, utilizando la técnica RNA-Seq por medio de la plataforma Illumina HiSeq. La base de datos en cuestión dispone de 801 instancias (una por cada paciente analizado) donde, a su vez, cada una de las instancias está definida por 20531 atributos (uno por cada gen sobre el que se analiza su nivel de expresión). Por su parte, las salidas deseadas se pueden etiquetar en las siguientes cinco clases:

- **BRCA**, **Br**east **Ca**ncer, del cual se dispone de 136 instancias.
- **KIRC**, **Ki**dney **R**enal Cell **C**arcinoma, del cual se dispone de 141 instancias.
- **COAD**, **Co**lon **Ad**enocarcinoma, del cual se dispone de 78 instancias.
- **LUAD**, **Lu**ng **Ad**enocarcinoma, del cual se dispone de 146 instancias.
- **PRAD**, **Pr**ostate **Ad**enocarcinoma, del cual se dispone de 300 instancias.

La ingente cantidad de información disponible para cada instancia y el carácter multiclase del problema, convierten en idóneo a este caso para la aplicación de las técnicas de aprendizaje máquina. Adicionalmente, se puede observar un desbalanceo entre el número de patrones que pertenecen a cada clase, lo cual resulta relevante a la hora de presentar los resultados. 

El dataset empleado está formado por dos ficheros. El primero de ellos es "data.csv" que contiene los datos de entrada al modelo donde las filas representan a cada una de las instancias y las columnas, sus correspondientes más de 20000 atributos (de 200 MB de tamaño). Por su parte, el fichero "labels.csv" contiene las correspondientes etiquetas de salida para cada uno de los patrones, es decir, a cuál de las cinco clases contempladas en el problema original pertenece. En la descripción del dataset dada por el repositorio, los genes presentes en "data.csv" son nombrados con una nomenclatura "dummy" por simplicidad, pero mantienen el orden de los nombres originales. A modo de resumen de las características del dataset, se adjunta la tabla que se provee en la propia página del repositorio.

<img src="./images/dataset_description.png">

Para presentar todos los pasos llevados a cabo con la intención de resolver el problema planteado, el presente documento presenta una sección de *Metodología*, donde se presentan tanto los pasos llevados a cabo para el entrenamiento de los modelos como los pasos previos. A continuación, en una sección de *Resultados y discusión* se presentarán los resultados obtenidos y una discusión sobre los mismos, estableciendo comparaciones entre diferentes aspectos a tener en cuenta. Finalmente, una sección de *Conclusiones* y otra de *Trabajo futuro*, donde se exponen las posibles líneas a seguir en posteriores aproximaciones al problema, concluyen la redacción de este documento.

# Metodología

A continuación, se enumerarán los pasos a llevar a cabo, para más tarde describir cada uno de ellos de forma más precisa.

- En primer lugar, para abordar el problema planteado, es necesario preprocesar los datos que provienen del dataset. Dado que el número de atributos es inmanejable de forma manual, primeramente es indispensable reducir la dimensionalidad de los datos además de aplicar una normalización de los mismos para favorecer el proceso de aprendizaje de los modelos. Una vez hecho, se procede a entrenar los modelos para más tarde poder evaluar su efectividad. Esto se describe con más detalle en la sección de *Lectura y preprocesamiento de los datos* que se muestra a continuación.

- En segundo lugar, una vez que los datos son correctamente procesados, se puede proceder a entrenar los modelos propuestos, obteniendo más tarde sus correspondientes resultados. Esto se describe con gran detalle en la sección *Entrenamiento de los modelos*.

Cabe destacar que el principal foco de este trabajo es observar la influencia del número de atributos obtenidos al aplicar las técnicas de preprocesamiento sobre la efectividad de los modelos. Por otra parte, se han seleccionado diferentes modelos de aprendizaje para estudiar la viabilidad de resolución del problema bajo diversos puntos de vista. Sin embargo no se pretende realizar un fine-tuning de los parámetros de dichos modelos. 

## Lectura y preprocesamiento de los datos

- Dado que el número de atributos es inmanejable de forma manual (20531), primeramente es indispensable reducir la dimensionalidad de los datos además de aplicar una normalización de los mismos para favorecer el proceso de aprendizaje de los modelos. 

- Para la reducción de la dimensionalidad, se contemplan dos opciones: utilizar algunas técnicas manuales como *features selection* o *features extraction* o bien la técnica estadística del *Principal Components Analysis* o *PCA*. Puesto que seleccionar un número determinado de atributos de entre los más de 20000 es súmamente complejo, consideramos que la técnica de PCA es mucho más adecuada para este problema concreto.

- Por otra parte, es necesario normalizar los datos de entrada para favorecer la tarea de aprendizaje por parte de los modelos. Para ello, se realiza una estandarización de los mismos en dos ocasiones: la primera, previa a la PCA y, la segunda, posterior a la aplicación de la misma (puesto que PCA podría desnormalizar los datos a la salida). 

In [ ]:
%%file read_data.py

import argparse
import numpy as np
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sys import argv

# En esta función se obtiene la matriz correspondiente al fichero .csv que
# se le pase como parámetro.
def get_data(filename):
    matrix = []
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            matrix.append(row)

    return np.array(matrix)

# Esta función es utilizada para almacenar datos en un fichero .csv.
def store_data(matrix, path):
    with open(path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        for row in range(0, np.shape(matrix)[0]):
            writer.writerow(matrix[row])

# Convertir la cadena de texto que representa la clase a la que pertenece un
# patrón en su vector de números correspondiente.
def get_numerical_output(tag_name):
    if (tag_name=='PRAD'):
        tag_number = [1, 0, 0, 0, 0]
    elif (tag_name=='LUAD'):
        tag_number = [0, 1, 0, 0, 0]
    elif (tag_name=='COAD'):
        tag_number = [0, 0, 1, 0, 0]
    elif (tag_name=='KIRC'):
        tag_number = [0, 0, 0, 1, 0]
    elif (tag_name=='BRCA'):
        tag_number = [0, 0, 0, 0, 1]
    else:
        tag_number = None

    return tag_number

def get_model_target(matrix):
    nrows = np.shape(matrix)[0]

    target = []
    for row in range(0, nrows):
        target.append(get_numerical_output(matrix[row]))

    return np.array(target)

def executePCA(input_matrix, args):
    n_components = args.pca_n
    pca = PCA(n_components)
    principal_components = pca.fit_transform(input_matrix)
    principal_components = StandardScaler().fit_transform(principal_components)

    return principal_components

def main(argv):
    input = get_data(argv.inputs_file)
    output = get_data(argv.targets_file)

    # Se obtiene el número de instancias de la base de datos.
    nofinstances = np.shape(input)[0]-1
    # Se obtiene el número de atributos de la base de datos.
    nofattributes = np.shape(input)[1]-1

    input = input[1:nofinstances+1, 1:nofattributes+1].astype(float)
    output = get_model_target(output[1:nofinstances+1, 1:])
    input = StandardScaler().fit_transform(input)
    principal_components = executePCA(input, argv)

    store_data(principal_components, argv.inputs_storing)
    store_data(output, argv.targets_storing)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Código de la práctica de bioinformática')
    parser.add_argument('--inputs_file', type=str)
    parser.add_argument('--targets_file', type=str)
    parser.add_argument('--pca_n', type=int)
    parser.add_argument('--inputs_storing', type=str)
    parser.add_argument('--targets_storing', type=str)
    argv = parser.parse_args()

    main(argv)


In [ ]:
%run read_data.py --inputs_file data.csv --targets_file labels.csv --pca_n 10 --inputs_storing input_storing.csv --targets_storing targets_storing.csv

La ejecución del script resulta en dos nuevos ficheros csv. Uno contendrá el resultado de la PCA sobre los datos de "data.csv", y otro, la matriz de targets formateada para realizar el entrenamiento.

## Entrenamiento de los modelos

Una vez los datos son convenientemente preprocesados, se procede a entrenar los tres modelos de aprendizaje máquina que se han seleccionado: algoritmo kNN, máquina de soporte vectorial y random forest. Dichos modelos han sido escogidos considerando su gran uso en inteligencia artificial para la resolución de problemas de clasificación y la adecuación que estos tienen a dichos problemas, por lo general. Para proceder al entrenamiento, es necesario realizar un splitting, es decir, en este caso concreto, dividir el conjunto de datos original en dos subconjuntos: el conjunto de entrenamiento y el conjunto de test. El splitting seleccionado para esta aproximación es de tipo aleatorio y con una repartición del 60%-40% para entrenamiento y test, respectivamente. Considerando ese elemento aleatorio, es necesario repetir la acción un número N de veces significativo. Sobre cada uno de los entrenamientos, se obtienen los valores de una serie de métricas. 

Posteriormente, para conseguir una métrica representativa de tal cantidad de entrenamientos, múltiples estadísticos son computados (en este caso, la media, para tener un valor concreto de la métrica y la desviación típica, para conocer la robustez de los modelos). Del mismo modo, es necesario escoger qué clase de métricas se desean utilizar para evaluar.

Cabe destacar, nuevamente, que el problema a tratar es de tipo multiclase y que posee concretamente cinco posibles etiquetas cuya descripción se expone en la sección de *Introducción*.

A modo de resumen, a continuación se exponen de forma breve los pasos a seguir:

- Selección del modelo a utilizar.
- Entrenamiento N veces de los modelos concretos realizando un splitting aleatorio diferente en cada una de las iteraciones del bucle.
- Registrar las métricas de bondad en cada uno de los ciclos del proceso de entrenamiento.
- Computar los estadísticos sobre los resultados obtenidos y así disponer de valores representativos para los N entrenamientos.

In [ ]:
%%file train_svm.py

import argparse
from sklearn import svm
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sys import argv

# En esta función se obtiene la matriz correspondiente al fichero .csv que
# se le pase como parámetro.
def get_data(filename):
    matrix = []
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            matrix.append(row)

    return np.array(matrix)

# Esta función se utiliza para convertir los vectores de la matriz target en
# un número entero.
def convert_vector_to_number(target_vector):
    if (target_vector[0]==1):
        output_value = 0
    if (target_vector[1]==1):
        output_value = 1
    if (target_vector[2]==1):
        output_value = 2
    if (target_vector[3]==1):
        output_value = 3
    if (target_vector[4]==1):
        output_value = 4

    return output_value

# Esta función se utiliza para convertir un número entero representativo de un
# tag en un vector.
def convert_number_to_vector(values):
    vectors = []
    for target_value in values:
        if (target_value==0):
            output_value = [1, 0, 0, 0, 0]
        if (target_value==1):
            output_value = [0, 1, 0, 0, 0]
        if (target_value==2):
            output_value = [0, 0, 1, 0, 0]
        if (target_value==3):
            output_value = [0, 0, 0, 1, 0]
        if (target_value==4):
            output_value = [0, 0, 0, 0, 1]
        vectors.append(output_value)

    return vectors

def process_targets(target_data):
    target_size = np.shape(target_data)

    final_targets = []
    for row in range(0, target_size[0]):
        final_targets.append(convert_vector_to_number(target_data[row, :]))

    return final_targets

def main(argv):
    input_data = get_data(argv.inputs_file).astype(float)
    target_data = get_data(argv.targets_file).astype(int)

    classifier = svm.SVC()
    target_data = process_targets(target_data)

    classifier.fit(input_data[0:500], target_data[0:500])
    prediction = classifier.predict(input_data[500:801])
    decision = classifier.decision_function(input_data[500:801])
    score = classifier.score(input_data[500:801], target_data[500:801])

    target = np.array(convert_number_to_vector(target_data[500:801]))

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    n_classes = 5

    plt.figure()
    colours = ['red', 'green', 'orange', 'brown', 'black']
    for it in range(0, n_classes):
        fpr[it], tpr[it], _ = roc_curve(target[:, it], decision[:, it])
        roc_auc[it] = auc(fpr[it], tpr[it])
        plt.plot(fpr[it], tpr[it], color=colours[it])

    plt.show()

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Código de la práctica de bioinformática')
    parser.add_argument('--inputs_file', type=str)
    parser.add_argument('--targets_file', type=str)
    argv = parser.parse_args()

    main(argv)

In [ ]:
%run train_svm.py --inputs_file input_storing.csv --targets_file targets_storing.csv

# Resultados y Discusión

A continuación, se presentan los resultados que se han obtenido con cada uno de los modelos en cuestión. En primer lugar, cabe destacar que los tiempos de ejecución de los experimentos, a pesar de realizarse cada uno N veces, se ejecutan en un tiempo relativamente aceptable. Destacar que, para todos los casos, el número de entrenamientos para el cual se computan los estadísticos deseados (media y desviación y desviación típica) es de 100.

### Algoritmo kNN

El parámetro usado para este modelo fue *k = 3*, de esta manera considerando los tres vecinos más próximos para realizar la clasificación. Cabe recordar que, al no realizar un fine-tuning de los parámetros que configuran este algoritmo, dicho valor se mantiene constante para todos los entrenamientos realizados. Se ejecutan cuatro experimentos diferentes, aplicando un PCA para obtener, respectivamente, 2, 3, 5 y 10 atributos. La principal característica que se observa es que un número más elevado de atributos comporta un incremento de la bondad de las métricas utilizadas, siendo el caso de 10 atributos el que ofrece unas métricas más aceptables. Para tener en cuenta la componente aleatoria que supone la partición del conjunto inicial en entrenamiento y test, se realizan 100 entrenamientos. Como métricas calculadas, se tiene la precisión global, la sensibilidad de cada una de las clases así como la AUC-ROC y la AUC-PR. Sin embargo, a efectos de comparación, se decide utilizar la precisión por ser una métrica global representativa de todas las clases y la AUC-PR que, para este caso, se ajusta mejor al problema puesto que esta métrica es adecuada cuando se trabaja con un conjunto desbalanceado en lo que respecta a número de patrones de cada clase. De hecho, observando los datos numéricos, podemos destacar que los valores de AUC-PR son menores o incluso significativamente menores que los de AUC-ROC o de la sensibilidad, lo cual destaca la relevancia de utilizar esta métrica como referencia.

Para entender la separación de los datos en el espacio, se utiliza primeramente como referencia al caso de utilizar una PCA que obtenga dos atributos. Se entrena el modelo y se observa cómo este es capaz de separar dichos datos, lo cual es observable en la imagen que se muestra a continuación. En ella se observa que, en un espacio de dos dimensiones, los datos no son fácilmente separables, de lo cual se intuye que no se trata de un número muy adecuado de atributos.

<img src="./images/values_2d_kNN.png">

En la imagen de abajo, se puede observar la comparativa entre los cuatro experimentos. Se trata de una gráfica donde se representa la media de las métricas seleccionadas y la desviación típica, de tal forma que la esquina superior izquierda representa el punto donde el clasificador obtiene una bondad máxima (ideal) para las métricas seleccionadas y, al mismo tiempo, donde dicho clasificador es más robusto (i.e. con una desviación típica pequeña, la variación de las métricas entre los N entrenamientos es muy pequeña y, por ello, el valor medio es más fiable). En el caso contrario, la esquina inferior derecha representa el extremo opuesto, teniendo en cuenta que la desviación típica es significativa (escasa robustez) y el valor medio de la métrica es muy bajo. Con respecto a la gráfica, los puntos simbolizan la precisión global y el resto de figuras representan los valores de AUC-PR para cada una de las clases. La finalidad de esta gráfica es simplemente estudiar a grandes rasgos cómo influye el número de atributos de entrada en los resultados y es por ello que no se hace énfasis en los valores concretos que se proporcionan para cada clase.  

<img src="./images/kNN_edited.png">

### Máquina de soporte vectorial (SVM)

Para este modelo, el kernel seleccionado es una función de base radial. Del mismo modo, como valor del parámetro C se escoge *C = 1.0*. Se realizan nuevamente los cuatro experimentos con PCA de 2, 3, 5 y 10 atributos, manteniendo constantes los dos parámetros anteriormente mencionados. Para cada experimento, con la intención de realizar una comparación en igualdad de condiciones, también se ejecutan 100 entrenamientos para más tarde computar la media y la desviación típica de las diferentes métricas cuantitativas utilizadas.

Para este caso, se vuelve a utilizar el ejemplo con dos atributos observando las divisiones que realiza el modelo de aprendizaje. El primer aspecto a destacar es que, igualmente, la separación de las instancias en clases le resulta muy compleja. Por otra parte, se observan diferencias significativas entre como el algoritmo kNN y la máquina de soporte vectorial realizan dichas separaciones.

<img src="./images/values_2d_svm.png">

En la imagen inferior, se pueden observar los resultados para este modelo de aprendizaje. Dichos resultados se interpretan del mismo modo que para el algoritmo kNN y se pueden concluir aspectos similares (un mayor número de atributos comporta una mejora de las métricas de evaluación en el punto de vista de su valor medio y una mayor robustez de los modelos entrenados). Destacar que los valores cuantitativos hacen referencia, de nuevo, a la precisión global y al AUC-PR de cada clase. 

<img src="./images/SVM.png">

### Random Forest

El último de los modelos utilizados ha sido random forest. Como parámetros, en primer lugar se ajusta el número de árboles de decisión que constituyen el bosque a 100. Por otra parte, la máxima profundidad para la expansión de los nodos en los mismos es de 4. Nuevamente, se realizan los mismos cuatro experimentos que con los algoritmos anteriores, ejecutando 100 entrenamientos para cada uno y de esta forma controlar el componente aleatorio del splitting. Las conclusiones extraídas, para este caso, son muy similares a las anteriores: mayor número de atributos implica modelos más precisos y robustos. Esto se puede observar en la imagen inferior, de nuevo mostrando las métricas de precisión global y de AUC-PR para cada clase.

En este último caso, el modelo es aparentemente incapaz de separar de forma conveniente los datos para obtener valores aceptables, del mismo modo que en los dos casos anteriores. También se observan diferencias significativas en cómo el modelo realiza la división de dicho espacio bidimensional.

<img src="./images/values_2d_rfor.png">

Cabe destacar que los entrenamientos de este tipo de modelos, para el problema aquí planteado, se muestran como los más pobres con respecto a la eficiencia en términos de tiempo, aunque continúan siendo tiempos muy aceptables.

<img src="./images/RFOR.png">

## Comparación de los modelos

- Una vez obtenidos los resultados para los tres tipos de modelos de aprendizaje empleados, buscando que todos ellos se puedan comparar en las mayores condiciones de igualdad posibles, se exponen conjuntamente en una cuarta gráfica. Para realizar esta comparación, se utilizan ya los resultados relativos únicamente a utilizar una PCA que obtenga 10 atributos a la salida, puesto que se considera como el mejor caso para los tres algoritmos. Las métricas utilizadas son, de nuevo, la precisión global y la AUC-PR por las mismas razones que se explican en apartados anteriores. 

- El primer detalle que se aprecia es que las métricas para el algoritmo kNN y para la máquina de soporte vectorial son muy parejos, donde el propio kNN parece ser ligeramente superior sobre todo en lo que respecta a la métrica de la precisión global.

- Por otra parte, el random forest parece ofrecer una efectividad más pobre dado que no solo la precisión global es más baja, si no que también los valores de AUC-PR para cada clase también parecen serlo.

Las comparaciones se pueden observar de forma visual en la imagen que se adjunta a continuación.

<img src="./images/comparison.png">

# Conclusiones

En el presente documento se ha expuesto un trabajo donde se utilizan técnicas de aprendizaje máquina para resolver un problema de clasificación en un dominio biomédico, tomando como partida una base de datos que representa el producto del análisis en la expresión de más de 20000 genes, utilizando la plataforma Illumina HiSeq y gracias a la técnica de RNA-Seq. Después de la consecución de múltiples experimentos tomando como referencia diversos modelos de aprendizaje máquina, se pueden extraer una serie de conclusiones:

- En primer lugar, la ingente cantidad de atributos de los que dispone cada una de las instancias (los más de 20000 genes) es inmanejable de forma manual. En estas circunstancias, el algoritmo de Principal Components Analysis o PCA se muestra como una de las aproximaciones más inmediatas e intuitivas de las que se dispone. A pesar de que los datos de salida no puedan ser comprendidos de forma intuitiva, ofrece grandes ventajas frente a otras estrategias como la de *features selection* o la de *features extraction*. 

- Por otra parte, cabe destacar la considerable cantidad de experimentos realizados, teniendo en cuenta que no solo se han utilizado tres modelos diferentes (algoritmo kNN, máquina de soporte vectorial y random forest) si no que además cada modelo ha sido probado con tres planteamientos diferentes: utilizando diferentes PCA que hayan obtenido, respectivamente, 2, 3, 5 y 10 atributos.

- Es digno de mencionar la utilización de un splitting aleatorio, para dividir el conjunto de patrones de entrada entre entrenamiento y test y, con ello, la necesidad de ejecutar N entrenamientos para cada experimento concreto, computando la media y la desviación típica para esos N entrenamientos, especificando así no solo un valor representativo de cada métrica empleada, si no también clarificando numéricamente la robustez de cada uno de los modelos. 

- En cuanto a la efectividad de los modelos, cabe decir que se calculan varias métricas, tomando como referencia la precisión global y la del valor de AUC-PR concreto para cada clase. En primer lugar, la precisión global proporciona una visión de efectividad para el problema en su conjunto en lugar de considerar cada clase de forma independiente; por su parte, la AUC-PR es una métrica adecuada para los problemas donde exista un desbalanceo en el número de patrones para cada una de las diferentes clases. 

- Con respecto a los resultados, se demuestra que el número de atributos utilizados a la entrada de los modelos de aprendizaje es relevante e influye de manera importante en su efectividad y robustez, observándose que los datos resultantes de aplicar una PCA que reduce los originales a 10 atributos son los que mejor ajustan el problema tal y como se ha tratado y se documenta en este texto, frente a casos donde el número de atributos sea inferior (2, 3 y 5, concretamente).

- Por su parte, en lo que se refiere a modelos de aprendizaje utilizados, la efectividad del algoritmo kNN es muy similar al de la máquina de soporte vectorial. En el polo opuesto se encuentra el random forest, el modelo que peores resultados ha mostrado que, aunque aceptables, son más pobres y los modelos obtenidos son menos robustos. 

- En general, se puede concluir que la forma de abordar el problema ha sido adecuada pero, como en cualquier trabajo de investigación, se pueden seguir tomando diversas líneas para evaluar el impacto de otros aspectos en los resultados.

# Trabajo futuro

Finalmente, tras la realización de los experimentos y la redacción de las principales conclusiones de los mismos, se pueden especificar algunas posibles líneas de expansión del presente trabajo en casos futuros.

- En primer lugar, la utilización PCA obteniendo un número de atributos mayor que 10, para estudiar en qué momento se alcanza un número idóneo de estos.

- Por otra parte, el planteamiento de un fine-tuning de los parámetros de los modelos utilizados.

- Otra posible extensión del trabajo consiste en utilizar diferentes modelos de aprendizajes a los ya empleados, como puede ser el caso de redes de neuronas artificiales, contemplando incluso la posibilidad de recurrir a arquitecturas de redes profundas.

- Del mismo modo, otra opción para línea futura supone aplicar técnicas como las de oversampling para resolver el problema del desequilibrio de patrones entre las diferentes clases que se mencionaba en apartados anteriores de este documento.